In [ ]:
%matplotlib inline
import os, sys, glob, scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nilearn, nibabel, nltools, nistats # , neurosynth
from nltools.data import Brain_Data
from nistats.reporting import get_clusters_table
from nilearn.plotting import plot_stat_map, plot_roi, plot_img, plot_glass_brain
from nilearn.input_data import NiftiMasker
import nibabel as nib

In [ ]:
base_dir = os.path.realpath('../../..')
print(base_dir)

## Function definitions

In [ ]:
def get_clusters(image, threshold = 0, extent_threshold = 0, sort_by_size = False, include_peaks_only = True):
    clusters = get_clusters_table(image, threshold, cluster_threshold = extent_threshold)
    clusters = clusters.rename(columns = {'Cluster ID':'ID', 'Cluster Size (mm3)':'size',
                                          'Peak Stat':'peak_value'})
    if include_peaks_only:
        peaks = clusters.copy()
        peaks = peaks.loc[peaks['ID'].apply(lambda x: not str(x)[-1].isalpha()),:].reset_index(drop=True)
#         peaks = peaks.sort_values(by = 'size', ascending = False).reset_index(drop=True)
    if sort_by_size:
        clusters = clusters.sort_values(by = 'size', ascending = False).reset_index(drop=True)
    return clusters, peaks

In [ ]:
def xyz_to_ijk(MNI, image):
    return list(np.linalg.inv(image.affine[:3,:3]).dot(MNI-image.affine[:3,3]).astype(int))

In [ ]:
def expand_cluster(image, cluster_peak_MNI):
    
    # Find IJK of peak
    i,j,k = xyz_to_ijk(cluster_peak_MNI,image)
    cluster_peak_IJK = [i,j,k]
    print('MNI: %s, IJK: %s'%(cluster_peak_MNI, cluster_peak_IJK))
    
    # Double check that peak value is correct
    peakval = image.get_data()[i][j][k]
    print('Peak value extracted from image data array: %f'%peakval)
    
    # Binarize image
    binarized = (image.get_data() != 0).astype(int)
        
    # Label each cluster with a different number, reserve 0 for empty voxels
    conn_mat = np.zeros((3, 3, 3), int)  # 6-connectivity, aka NN1 or "faces"
    conn_mat[1, 1, :] = 1
    conn_mat[1, :, 1] = 1
    conn_mat[:, 1, 1] = 1
    label_map = scipy.ndimage.measurements.label(binarized, conn_mat)[0]
    clust_image = nibabel.Nifti1Image(label_map, affine=image.affine)
    
    # Find voxels with same label as cluster peak
    cluster_label = clust_image.get_data()[i,j,k]
    cluster_ROI = (clust_image.get_data() == cluster_label).astype(int)
    ROI_mask = nibabel.Nifti1Image(cluster_ROI, affine = image.affine)
    print('Cluster ROI located')
    print('Cluster size as extracted from image data array: %i voxels = %i mm^3'%(
        sum(cluster_ROI.flatten()),sum(cluster_ROI.flatten())*8))
    
    return cluster_peak_IJK, cluster_ROI, ROI_mask

## Select effect of interest

In [ ]:
run = 3
filter_TR = False
TR_start = 1
TR_end = 711
use_anova_out = False
model = 'ideology_IUS'
term = 'scale(ideology_similarity)-X-joint_IUS'
statistic = 'beta' if use_anova_out == False else 'F'
threshold = 'thr-pval-fdr-0.05'

In [ ]:
suffix = '_with_anova' if use_anova_out else ''
results_dir = base_dir + '/Results/voxelwise_ISC/nifti' + suffix
run_model_dir = ('run-%i_TRs-%i-%i_model-%s'%(run,TR_start,TR_end,model) 
                     if filter_TR else 'run-%i_model-%s'%(run,model))
fpath = glob.glob('%s/%s/*%s*%s*.nii.gz'%(results_dir, run_model_dir, term, threshold))[0]

In [ ]:
stat_path = glob.glob('%s/%s/*%s*%s.nii.gz'%(results_dir, run_model_dir, term, statistic))[0]
stat_path

In [ ]:
p_path = beta_path = glob.glob('%s/%s/*%s*%s.nii.gz'%(results_dir, run_model_dir, term, 'pval'))[0]
p_path

In [ ]:
stat_thresh_path = glob.glob('%s/%s/*%s*%s%s.nii.gz'%(results_dir, run_model_dir, term, statistic, '-thr-pval-fdr-0.05'))[0]
stat_thresh_path

In [ ]:
stat_map, p_map, stat_thresh_map = [nib.load(path) for path in [stat_path, p_path, stat_thresh_path]]

In [ ]:
%matplotlib inline
plot_stat_map(stat_thresh_map)
plt.show()

## Find activation clusters in tresholded stat map

In [ ]:
clusters, peaks = get_clusters(stat_thresh_map, 0, 5)
print('%i clusters found, loading...\n..\n.\n'%peaks.shape[0])
cluster_list = peaks['ID'].unique()
ROIs = dict()
all_cluster_info = pd.DataFrame()
for ID in cluster_list:
    print(ID)
    cluster_info = peaks.query('ID == @ID').copy()
    cluster_peak_MNI = list(cluster_info.iloc[0].loc[['X','Y','Z']].values.flatten())
    i,j,k = xyz_to_ijk(cluster_peak_MNI, p_map)
    peak_p = p_map.get_data()[i,j,k]
    cluster_info['peak_p'] = peak_p
    cluster_info['size (nvox)'] = cluster_info['size']/27
    display(cluster_info)
    all_cluster_info = all_cluster_info.append(
        cluster_info[['ID','X','Y','Z','size (nvox)','peak_value','peak_p']]).reset_index(drop=True)

Put the names in manually as a dict, e.g.:

In [ ]:
# cluster_names = {1:'rFEF',2:'lTPJ',3:'rMT',4:'rOFC',5:'lMT',6:'precuneus',7:'rTPJ',8:'lFEF',9:'lFEF',10:'precuneus'}

In [ ]:
all_cluster_info['Region name'] = all_cluster_info['ID'].map(cluster_names)
all_cluster_info = all_cluster_info.sort_values(by=['size (nvox)','peak_value'], ascending = False
                                               ).reset_index(drop=True)
all_cluster_info = all_cluster_info[['ID','Region name','size (nvox)','X','Y','Z','peak_value','peak_p']]
all_cluster_info.head()

In [ ]:
all_cluster_info.to_csv(results_dir + '/' + run_model_dir + 
                        '/ROI_list_term-%s.csv'%term)